In [2]:
import os
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon
import plotly.offline as py#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

In [1]:
import json

tweets_file = open('geotagged_tweets_20160812-0912.1000.jsons', "r")

def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []

# This still needs some work!!
# Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)
        tweets_list.append(tweet_obj)
    return tweets_list

tweets = flatten_tweets(tweets_file)


In [3]:
import pandas as pd

from pandas.io.json import json_normalize
# Thank gawd for this package - all json fields straight into columns in a dataframe, no bullshit
tweets_df = json_normalize(tweets)


In [17]:
cols = []

for col in tweets_df.columns:
    if col in ('created_at', 'place.country','place.country_code','place.name','is_quote_status','lang','text','user.created_at','user.description','user.name','user.location','place.place_type', 'user-screen_name', 'in_reply_to_screen_name','geo.type','geo.coordinates', 'retweeted_status-text'):
        cols.append(col)

neat_df = tweets_df[cols]
neat_df.head()

,created_at,text,in_reply_to_screen_name,is_quote_status,lang,user.name,user.location,user.description,user.created_at,place.place_type,place.name,place.country_code,place.country,geo.type,geo.coordinates
0,Sun Sep 11 02:05:23 +0000 2016,@HillaryClinton @POTUS @JudgeJeanine @NBA @NFL...,HillaryClinton,False,en,RonaldSouthwell,"Evansville, IN",#Election2016 #Election2016,Fri Jun 24 13:46:12 +0000 2016,city,Evansville,US,United States,NaN,NaN
1,Wed Aug 17 20:18:54 +0000 2016,IF THEY ONLY HAD A BRAIN! \n#NeverTrump #BuhBy...,None,False,en,Jimmy Bear,"Nutley, New Jersey",#TeamGay.#Atheist.HATE is NOT a Family Value.J...,Mon Aug 15 12:09:02 +0000 2011,city,Nutley,US,United States,NaN,NaN
2,Fri Sep 02 23:06:23 +0000 2016,Attorney's first line of defense to those they...,None,False,en,Pops,"Oklahoma City, OK",Patriot. Constitutionalist. Conservative. Anti...,Wed Aug 22 14:38:00 +0000 2012,city,Oklahoma City,US,United States,NaN,NaN
3,Thu Sep 01 03:01:51 +0000 2016,@kikesma @norma22flores @EPN @realDonaldTrump ...,kikesma,False,es,Alonso Betancurt,None,None,Wed May 21 21:39:25 +0000 2014,city,Villaldama,MX,México,NaN,NaN
4,Sat Sep 10 05:22:45 +0000 2016,Classy.. Better than I expected. But on a se...,None,True,en,William McCarthy,"Massachusetts, USA",Kasich supporter. never ever trump. trump is j...,Tue Feb 23 03:13:22 +0000 2016,city,Marblehead,US,United States,NaN,NaN


In [26]:
import re


#probably gotta clean it more but this is a first step
def clean_text(dataframe, col):
    # make new column for accounts
    dataframe["accounts"] = dataframe[col].str.findall(r'@.*?(?=\s|$)')
    # make new column for hashtag
    dataframe["hashtag"] = dataframe[col].str.findall(r'#.*?(?=\s|$)')
    #remove @
    dataframe[col] = dataframe[col].str.replace(r'@.*?(?=\s|$)'," ")
    #remove #
    dataframe[col] = dataframe[col].str.replace(r'#.*?(?=\s|$)'," ")
    #remove links
    dataframe[col] = dataframe[col].apply(lambda x:re.split('https:\/\/.*',str(x))[0])
    return dataframe

clean_df = clean_text(neat_df, "text")
clean_df.head(n = 26)

,created_at,text,in_reply_to_screen_name,is_quote_status,lang,user.name,user.location,user.description,user.created_at,place.place_type,place.name,place.country_code,place.country,geo.type,geo.coordinates,accounts,hashtag
0,Sun Sep 11 02:05:23 +0000 2016,There IS A PLAN U PIC ❤️✝👀,HillaryClinton,False,en,RonaldSouthwell,"Evansville, IN",#Election2016 #Election2016,Fri Jun 24 13:46:12 +0000 2016,city,Evansville,US,United States,NaN,NaN,"[@HillaryClinton, @POTUS, @JudgeJeanine, @NBA,...",[]
1,Wed Aug 17 20:18:54 +0000 2016,IF THEY ONLY HAD A BRAIN! \n \n,None,False,en,Jimmy Bear,"Nutley, New Jersey",#TeamGay.#Atheist.HATE is NOT a Family Value.J...,Mon Aug 15 12:09:02 +0000 2011,city,Nutley,US,United States,NaN,NaN,[],"[#NeverTrump, #BuhByeTRUMP, #TrumpIsUnfit]"
2,Fri Sep 02 23:06:23 +0000 2016,Attorney's first line of defense to those they...,None,False,en,Pops,"Oklahoma City, OK",Patriot. Constitutionalist. Conservative. Anti...,Wed Aug 22 14:38:00 +0000 2012,city,Oklahoma City,US,United States,NaN,NaN,[@HillaryClinton],[#lyingHillary]
3,Thu Sep 01 03:01:51 +0000 2016,No lo recibió como presidente,kikesma,False,es,Alonso Betancurt,None,None,Wed May 21 21:39:25 +0000 2014,city,Villaldama,MX,México,NaN,NaN,"[@kikesma, @norma22flores, @EPN, @realDonaldTr...",[]
4,Sat Sep 10 05:22:45 +0000 2016,Classy.. Better than I expected. But on a se...,None,True,en,William McCarthy,"Massachusetts, USA",Kasich supporter. never ever trump. trump is j...,Tue Feb 23 03:13:22 +0000 2016,city,Marblehead,US,United States,NaN,NaN,[],[]
5,Sat Aug 27 04:28:17 +0000 2016,I wouldn't even consider it when it is a con...,realDonaldTrump,False,en,Rob Desmond,"Santa Clarita Valley, CA",That the poweful play goes on and you may cont...,Tue Jan 03 02:43:33 +0000 2012,city,Stevenson Ranch,US,United States,NaN,NaN,[@realDonaldTrump],[]
6,Thu Aug 25 15:42:29 +0000 2016,"Join me again 11:45am re dumb ""bigot"" co...",None,False,en,Maria Cardona,Washington DC,"Political Strategist, CNN/CNNEE Contributor, P...",Fri Mar 19 15:26:01 +0000 2010,city,Washington,US,United States,NaN,NaN,"[@CNN, @ThisHour]",[#Trump's]
7,Wed Aug 17 19:36:22 +0000 2016,More fraud from soros and Clinton,None,True,en,Cindy zarember,"Los Alamitos, CA",Business Owner BIG Trump supporter member of ...,Wed Mar 12 00:24:06 +0000 2014,city,Los Alamitos,US,United States,NaN,NaN,[],[]
8,Sun Aug 21 19:59:25 +0000 2016,Right! thinks he's special‼️,None,True,en,DEBI129,None,CLINTON❤️KAINE ❤️BLM❤️EQUALITY❤️PP❤️ NO ABUSER...,Thu Sep 25 01:28:59 +0000 2008,admin,Washington,US,United States,NaN,NaN,[@realDonaldTrump],[]
9,Tue Aug 23 05:20:59 +0000 2016,I want to say that he will prosecute her a...,JohnKStahlUSA,False,en,SWalker for Trump,USA,Tired of @GOP not delivering on promises or le...,Mon Nov 03 17:55:38 +0000 2008,city,Houston,US,United States,NaN,NaN,"[@JohnKStahlUSA, @realDonaldTrump]",[]


In [27]:
def label_language(df) :
    if df["lang"] == "en" :
        return "English"
    elif df["lang"] == "es" :
        return "Spanish"
    elif df["lang"] == "ger":
        return "German"
    elif df["lang"] == "nl" :
        return "Dutch"
    elif df["lang"] == "it" :
        return "Italian"
    elif df["lang"] == "fr":
        return "French"
    else :
        return "Other"
neat_df["lang"] = neat_df.apply(lambda neat_df:label_language(neat_df),axis = 1)

In [28]:
neat_df.head(n = 10)

,created_at,text,in_reply_to_screen_name,is_quote_status,lang,user.name,user.location,user.description,user.created_at,place.place_type,place.name,place.country_code,place.country,geo.type,geo.coordinates,accounts,hashtag
0,Sun Sep 11 02:05:23 +0000 2016,There IS A PLAN U PIC ❤️✝👀,HillaryClinton,False,English,RonaldSouthwell,"Evansville, IN",#Election2016 #Election2016,Fri Jun 24 13:46:12 +0000 2016,city,Evansville,US,United States,NaN,NaN,"[@HillaryClinton, @POTUS, @JudgeJeanine, @NBA,...",[]
1,Wed Aug 17 20:18:54 +0000 2016,IF THEY ONLY HAD A BRAIN! \n \n,None,False,English,Jimmy Bear,"Nutley, New Jersey",#TeamGay.#Atheist.HATE is NOT a Family Value.J...,Mon Aug 15 12:09:02 +0000 2011,city,Nutley,US,United States,NaN,NaN,[],"[#NeverTrump, #BuhByeTRUMP, #TrumpIsUnfit]"
2,Fri Sep 02 23:06:23 +0000 2016,Attorney's first line of defense to those they...,None,False,English,Pops,"Oklahoma City, OK",Patriot. Constitutionalist. Conservative. Anti...,Wed Aug 22 14:38:00 +0000 2012,city,Oklahoma City,US,United States,NaN,NaN,[@HillaryClinton],[#lyingHillary]
3,Thu Sep 01 03:01:51 +0000 2016,No lo recibió como presidente,kikesma,False,Spanish,Alonso Betancurt,None,None,Wed May 21 21:39:25 +0000 2014,city,Villaldama,MX,México,NaN,NaN,"[@kikesma, @norma22flores, @EPN, @realDonaldTr...",[]
4,Sat Sep 10 05:22:45 +0000 2016,Classy.. Better than I expected. But on a se...,None,True,English,William McCarthy,"Massachusetts, USA",Kasich supporter. never ever trump. trump is j...,Tue Feb 23 03:13:22 +0000 2016,city,Marblehead,US,United States,NaN,NaN,[],[]
5,Sat Aug 27 04:28:17 +0000 2016,I wouldn't even consider it when it is a con...,realDonaldTrump,False,English,Rob Desmond,"Santa Clarita Valley, CA",That the poweful play goes on and you may cont...,Tue Jan 03 02:43:33 +0000 2012,city,Stevenson Ranch,US,United States,NaN,NaN,[@realDonaldTrump],[]
6,Thu Aug 25 15:42:29 +0000 2016,"Join me again 11:45am re dumb ""bigot"" co...",None,False,English,Maria Cardona,Washington DC,"Political Strategist, CNN/CNNEE Contributor, P...",Fri Mar 19 15:26:01 +0000 2010,city,Washington,US,United States,NaN,NaN,"[@CNN, @ThisHour]",[#Trump's]
7,Wed Aug 17 19:36:22 +0000 2016,More fraud from soros and Clinton,None,True,English,Cindy zarember,"Los Alamitos, CA",Business Owner BIG Trump supporter member of ...,Wed Mar 12 00:24:06 +0000 2014,city,Los Alamitos,US,United States,NaN,NaN,[],[]
8,Sun Aug 21 19:59:25 +0000 2016,Right! thinks he's special‼️,None,True,English,DEBI129,None,CLINTON❤️KAINE ❤️BLM❤️EQUALITY❤️PP❤️ NO ABUSER...,Thu Sep 25 01:28:59 +0000 2008,admin,Washington,US,United States,NaN,NaN,[@realDonaldTrump],[]
9,Tue Aug 23 05:20:59 +0000 2016,I want to say that he will prosecute her a...,JohnKStahlUSA,False,English,SWalker for Trump,USA,Tired of @GOP not delivering on promises or le...,Mon Nov 03 17:55:38 +0000 2008,city,Houston,US,United States,NaN,NaN,"[@JohnKStahlUSA, @realDonaldTrump]",[]
